<a href="https://colab.research.google.com/github/ShinjiNeo/Test_ML/blob/main/Template_Classificazione_Multiclass_PIPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template per classificazione Multiclass PIPE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer,OneHotEncoder,StandardScaler,FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

In [3]:
path = 'https://frenzy86.s3.eu-west-2.amazonaws.com/python/penguins.csv'
df = pd.read_csv(path, sep=',')
#df = pd.read_csv(path,)
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181,3750,male
1,Adelie,Torgersen,39.5,17.4,186,3800,female
2,Adelie,Torgersen,40.3,18.0,195,3250,female
3,Adelie,Torgersen,36.7,19.3,193,3450,female
4,Adelie,Torgersen,39.3,20.6,190,3650,male
...,...,...,...,...,...,...,...
328,Chinstrap,Dream,55.8,19.8,207,4000,male
329,Chinstrap,Dream,43.5,18.1,202,3400,female
330,Chinstrap,Dream,49.6,18.2,193,3775,male
331,Chinstrap,Dream,50.8,19.0,210,4100,male


In [ ]:
# df.columns = [''] # per rinominare eventualmente le colonne
# df

- Colonne da droppare

In [4]:
# Io non dropperei il sesso, ma in questo caso lo faccio ugualmente seguendo chi l'ha fatto
features_to_remove = ['sex']
df = df.drop(features_to_remove, axis=1)
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,Torgersen,39.1,18.7,181,3750
1,Adelie,Torgersen,39.5,17.4,186,3800
2,Adelie,Torgersen,40.3,18.0,195,3250
3,Adelie,Torgersen,36.7,19.3,193,3450
4,Adelie,Torgersen,39.3,20.6,190,3650
...,...,...,...,...,...,...
328,Chinstrap,Dream,55.8,19.8,207,4000
329,Chinstrap,Dream,43.5,18.1,202,3400
330,Chinstrap,Dream,49.6,18.2,193,3775
331,Chinstrap,Dream,50.8,19.0,210,4100


## Separo Features da Target

In [5]:
X = df.drop(['species'], axis=1) # matrice Features
y = df['species']

In [6]:
X.columns

Index(['island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm',
       'body_mass_g'],
      dtype='object')

In [7]:
# Divido le Categoriche da quelle Numeriche
numerical_features = [x for x, dtype in zip(X.columns, X.dtypes) if dtype.kind in ['i','f'] ]
categorical_features = [x for x, dtype in zip(X.columns, X.dtypes) if dtype.kind not in ['i','f']]

print('Numerical : ' + str(numerical_features))
print('Categorical : ' + str(categorical_features))

Numerical : ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
Categorical : ['island']


## Pre Processing in PIPE

In [8]:
# qui non sarebbe necessario farlo, dato che i dati sono puliti, ma lo metto comunque per essere più comodo dopo

## Parte di processingdata: Cleaning, fill Nan + OHE
Preprocessor = make_column_transformer(
                                        (make_pipeline(
                                        SimpleImputer(strategy = 'median'),
                                        KBinsDiscretizer(n_bins=3)), numerical_features),

                                        (make_pipeline(
                                        #FunctionTransformer(lambda x: np.where(x == 'n.d.', np.nan, x)),
                                        SimpleImputer(strategy = 'constant', fill_value = 'missing'),
                                        OneHotEncoder(categories = 'auto', handle_unknown = 'ignore',drop='first')), categorical_features)
                                        )

## Standardizzazione

In [9]:
scaler = StandardScaler()



## Applico Best Fit al classificatore scelto

In [10]:
classifier = RandomForestClassifier(bootstrap=True,
                                    max_depth=80,
                                    max_features=2,
                                    min_samples_leaf=3,
                                    min_samples_split=8,
                                    n_estimators=100
                                    )
# classifier = LogisticRegression()

## Pipe

In [11]:
model_pipe = Pipeline([
                       ('Preprocessing features', Preprocessor),
                       ('Scaling and standadize data', scaler),
                       ('Classifier', classifier)
                      ])
model_pipe

Pipeline(steps=[('Preprocessing features',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('kbinsdiscretizer',
                                                                   KBinsDiscretizer(n_bins=3))]),
                                                  ['bill_length_mm',
                                                   'bill_depth_mm',
                                                   'flipper_length_mm',
                                                   'body_mass_g']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['island'])])),
                ('Scaling and standadize data', StandardScaler()),
                ('Classifier',
                 RandomForestClassifier(max_depth=80, max_features=2,
                                        min_samples_leaf=3,
                                        min_samples_split=8))])

- Cross validation di quello che ho fatto finora

In [12]:
cross_val_scores = cross_val_score(model_pipe, X, y, cv=5)
print(cross_val_scores)

# Calcola la media e la deviazione standard dei punteggi
mean_score = np.mean(cross_val_scores)
std_score = np.std(cross_val_scores)

print(f"Mean cross-validation score:{mean_score:.2f}")
print(f"Standard deviation of cross-validation scores:{std_score:.2f}")

[0.92537313 0.98507463 0.95522388 1.         0.96969697]
Mean cross-validation score:0.97
Standard deviation of cross-validation scores:0.03


- fitto

In [13]:
model_pipe.fit(X, y)

Pipeline(steps=[('Preprocessing features',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('kbinsdiscretizer',
                                                                   KBinsDiscretizer(n_bins=3))]),
                                                  ['bill_length_mm',
                                                   'bill_depth_mm',
                                                   'flipper_length_mm',
                                                   'body_mass_g']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['island'])])),
                ('Scaling and standadize data', StandardScaler()),
                ('Classifier',
                 RandomForestClassifier(max_depth=80, max_features=2,
                                        min_samples_leaf=3,
                                        min_samples_split=8))])

In [14]:
y_pred_tot = model_pipe.predict(X)

In [15]:
from sklearn.metrics import classification_report

classification_report(y,y_pred_tot )

classification_report_result = classification_report(y, y_pred_tot)
print(classification_report_result)

              precision    recall  f1-score   support

      Adelie       0.98      0.98      0.98       146
   Chinstrap       0.96      0.94      0.95        68
      Gentoo       0.99      1.00      1.00       119

    accuracy                           0.98       333
   macro avg       0.98      0.97      0.97       333
weighted avg       0.98      0.98      0.98       333



## Predizione di una singola osservazione

In [22]:
X.columns

Index(['island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm',
       'body_mass_g'],
      dtype='object')

In [16]:
island = "Dream"
bill_length_mm = 50.0
bill_depth_mm = 20.0
flipper_length_mm = 200.0
body_mass_g = 4000.0

data = {
        "island": [island],
        "bill_length_mm": [bill_length_mm],
        "bill_depth_mm": [bill_depth_mm],
        "flipper_length_mm": [flipper_length_mm],
        "body_mass_g": [body_mass_g]
        }

input_df = pd.DataFrame(data)
res = model_pipe.predict(input_df)#.astype(int)[0]
print(res)

# classes = {0:'died',
#            1:'survived',
#            2:''
#            }

#### y_pred_tot = classes[res]
#### y_pred_tot

['Chinstrap']


## Imposto dove salvare il modello PKL

In [17]:
cd/content/drive/MyDrive/Machine_Learning/Template_Esame_MLearning/REG_CLASSIFICAZIONE

/content/drive/MyDrive/Machine_Learning/Template_Esame_MLearning/REG_CLASSIFICAZIONE


- importo joblib

In [18]:
import joblib

joblib.dump(model_pipe,'penguins_pipe.pkl')

['penguins_pipe.pkl']

- carico joblib

In [19]:
import joblib

model_pipe = joblib.load('penguins_pipe.pkl')
print('Model loaded successfully')

Model loaded successfully


- vedo cosa c'è nella PIPE

In [20]:
model_pipe

Pipeline(steps=[('Preprocessing features',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('kbinsdiscretizer',
                                                                   KBinsDiscretizer(n_bins=3))]),
                                                  ['bill_length_mm',
                                                   'bill_depth_mm',
                                                   'flipper_length_mm',
                                                   'body_mass_g']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['island'])])),
                ('Scaling and standadize data', StandardScaler()),
                ('Classifier',
                 RandomForestClassifier(max_depth=80, max_features=2,
                                        min_samples_leaf=3,
                                        min_samples_split=8))])

- Predizione

In [21]:
island = "Dream"
bill_length_mm = 50.0
bill_depth_mm = 20.0
flipper_length_mm = 200.0
body_mass_g = 4000.0

data = {
        "island": [island],
        "bill_length_mm": [bill_length_mm],
        "bill_depth_mm": [bill_depth_mm],
        "flipper_length_mm": [flipper_length_mm],
        "body_mass_g": [body_mass_g]
        }

input_df = pd.DataFrame(data)
res = model_pipe.predict(input_df)#.astype(int)[0]
print(res)

# classes = {0:'died',
#            1:'survived',
#            2:''
#            }

#### y_pred_tot = classes[res]
#### y_pred_tot

['Chinstrap']
